In [1]:
from packages import *
import pandas as pd
import tensorflow as tf
from datetime import datetime, timedelta
import time
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.data.experimental import AUTOTUNE
from sklearn.metrics import f1_score, r2_score
from sklearn.model_selection import train_test_split
from collections import Counter
%load_ext autoreload
%autoreload 2

# Model
We will be building our models in this notebook.

Some configuration required for tensorflow.

In [2]:
tf.config.gpu.set_per_process_memory_growth(True)
tf.config.gpu.set_per_process_memory_fraction(.90)
tf.keras.backend.set_image_data_format('channels_last')

## Dataset

In [3]:
batch_size = 64
glove_dim = 300
input_shape = (300, glove_dim, 1)
max_length = 300
shuffle_buffer_size = batch_size * 10
prefetch_buffer_size = 1
chunk_count = 100
random_seed = np.random.randint(0, 1000)
splits = ['train', 'val', 'test'] #80:10:10

In [4]:
tf_text_filenames = [os.path.join(DATASET_DIR, 'preprocessed', 'tfrecord', 'xext',  #xext temporarily
                                  'review-text-{:02d}.tf'.format(i)) for i in range(chunk_count)]
tf_ix_filenames = [os.path.join(DATASET_DIR, 'preprocessed', 'tfrecord', 'ix', 
                                  'review-ix-{:02d}.tf'.format(i)) for i in range(chunk_count)]
stars = (load_data({'review': ['stars']})['review']['stars'] - 1).astype(np.int32)# this is so that stars are 0-indexed
N = len(stars)
stars_chunked = stars.reshape((chunk_count, 6685900//chunk_count))

In [5]:
tf_text, tf_ix, stars= {}, {}, {}

In [6]:
tf_text['train'], tf_text_filenames_val_test, tf_ix['train'], tf_ix_filenames_val_test, stars['train'], stars_val_test \
= train_test_split(tf_text_filenames, tf_ix_filenames, stars_chunked, random_state=random_seed, test_size = .2)

In [7]:
tf_text['val'], tf_text['test'], tf_ix['val'], tf_ix['test'], stars['val'], stars['test'] \
= train_test_split(tf_text_filenames_val_test, tf_ix_filenames_val_test, stars_val_test, random_state=random_seed, test_size = .5)

In [8]:
for split in splits:
    stars[split] = stars[split].reshape(np.product(stars[split].shape))

In [9]:
print('({}:{}:{})'.format(len(tf_text['train']), len(tf_text['val']), len(tf_text['test'])))

(80:10:10)


In [10]:
glove_lookup = load_pickle(os.path.join(GLOVE_DIR, 'glove-{}D-byte-float32.pkl'.format(glove_dim)))
keys_to_ix = load_pickle(os.path.join(GLOVE_DIR, 'glove-byte-keys_to_ix.pkl'))
ix_to_key = {value: key for key, value in keys_to_ix.items()}

In [11]:
unk_vector = np.mean(np.array(list(glove_lookup.values())), axis=0)
null_vector = np.zeros(glove_dim)
glove_lookup[UNK_KEY.encode('ascii')] = unk_vector #if using byte glove dict
glove_lookup[NULL_KEY.encode('ascii')] = null_vector

In [12]:
oops = 0
glove_lookup_array = []
for i in range(len(ix_to_key)):
    if ix_to_key[i] not in glove_lookup:
        oops+=1
    glove_lookup_array.append(glove_lookup.get(ix_to_key[i], null_vector))
glove_lookup_array = np.array(glove_lookup_array, dtype=np.float32)
print('oops {} times.'.format(oops))

oops 1 times.


In [13]:
@tf.function
def _parse_function(proto, to_ix):
    # define your tfrecord again. Remember that you saved your image as a string.
    keys_to_features = {'review': tf.io.FixedLenFeature([300,], tf.int64) if to_ix else tf.io.FixedLenFeature([300,], tf.string),}
    
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    return parsed_features 

In [14]:
# tf_embedding = tf.Variable(
#         tf.constant(glove_lookup_array),
#         trainable=False,
#         name="Embedding"
# )

In [15]:
@tf.function
def embed(tensor):
    return tf.gather(glove_lookup_array, tensor)

In [16]:
@tf.function
def channelize(tensor):
    return tf.expand_dims(tensor, -1)

In [17]:
def make_dataset(embedded, to_ix=False):        
    if embedded or to_ix: 
        files = tf_ix
        to_ix = True
    else:
        files = tf_text
    dataset = {}
    for split in splits:
        dataset[split] = tf.data.TFRecordDataset(files[split])
        stars_dataset = tf.data.Dataset.from_tensor_slices(stars[split])
        dataset[split] = dataset[split].map(lambda x: _parse_function(x, to_ix)['review'], num_parallel_calls=AUTOTUNE)
        if embedded:
            dataset[split] = dataset[split].map(embed, num_parallel_calls=AUTOTUNE)
            dataset[split] = dataset[split].map(channelize, num_parallel_calls=AUTOTUNE)
        dataset[split] = tf.data.Dataset.zip((dataset[split], stars_dataset))
        dataset[split] = dataset[split].shuffle(shuffle_buffer_size)
        dataset[split] = dataset[split].map(lambda x, y: ({'review': x}, {'stars': y}))
        dataset[split] = dataset[split].batch(batch_size)
        dataset[split] = dataset[split].prefetch(prefetch_buffer_size)
    return dataset

## Models

### JingleBell

In [18]:
class JingleBell:
    
    def build(star_units=5):
        inputs = Input(shape=(max_length, glove_dim), name='review')
        x = Flatten(name='flatten')(inputs)
        out = Dense(5, activation='softmax', name='stars', kernel_regularizer='l1')(x)
        return tf.keras.models.Model(inputs={'review':inputs}, outputs={'stars': out}, name='JingleBell')


In [19]:
dataset = make_dataset(embedded=True)

In [29]:
dataset['test'].keys()

AttributeError: 'PrefetchDataset' object has no attribute 'keys'

In [20]:
bell = JingleBell.build()

In [21]:
Z=[]
for x in dataset['train'].take(1000):
    Z.append(x)

In [22]:
for i in range(len(X)):
    assert(np.array_equal(X[i][1]['stars'], Z[i][1]['stars']))

NameError: name 'X' is not defined

In [23]:
bell.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
log_dir = os.path.join(SRC_DIR, 'logs', '{}'.format(datetime.fromtimestamp(time.time()).strftime('%H-%M-%S_%Y-%m-%d')))
tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=False, update_freq='batch')

In [25]:
bell.fit(dataset['train'].take(1000), epochs=10, validation_data= dataset['val'].take(100), callbacks=[tb])

Epoch 1/10
1000/1000 [==============================] - 22s 22ms/step - loss: 3.0160 - accuracy: 0.4414 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
1000/1000 [==============================] - 21s 21ms/step - loss: 2.8733 - accuracy: 0.4424 - val_loss: 2.9236 - val_accuracy: 0.4317
Epoch 3/10
1000/1000 [==============================] - 21s 21ms/step - loss: 2.8671 - accuracy: 0.4428 - val_loss: 2.9224 - val_accuracy: 0.4322
Epoch 4/10
1000/1000 [==============================] - 21s 21ms/step - loss: 2.8645 - accuracy: 0.4426 - val_loss: 2.9247 - val_accuracy: 0.4323
Epoch 5/10
1000/1000 [==============================] - 21s 21ms/step - loss: 2.8627 - accuracy: 0.4421 - val_loss: 2.9155 - val_accuracy: 0.4316
Epoch 6/10
1000/1000 [==============================] - 21s 21ms/step - loss: 2.8615 - accuracy: 0.4425 - val_loss: 2.9175 - val_accuracy: 0.4325
Epoch 7/10
1000/1000 [==============================] - 21s 21ms/step - loss: 2.8607 - accuracy: 0.4425 - val_loss: 

### BabyBlueberry

### BabyBlueberry
Convolutional Model with GLoVe embeddings.

In [18]:
class BabyBlueberry:

    def build(input_shape=(max_length, glove_dim, 1), star_units=5):
        
        inputs = Input(shape=input_shape, name='review')
        
        lstm = LSTM(10, name="LSTM")(inputs)
        
        x_3 = Conv2D(300, (3, glove_dim), activation='relu', name='conv_3', padding='valid', kernel_regularizer='l2')(inputs)
#         x_3 = GlobalMaxPooling2D(name='pool_3')(x_3)
        x_3 = MaxPooling2D(pool_size=(max_length - 2, 1), name='pool_3')(x_3)

        
        
        x_4 = Conv2D(300, (4, 300), activation='relu', name='conv_4', padding='valid', kernel_regularizer='l2')(inputs)
#         x_4 = GlobalMaxPooling2D(name='pool_4')(x_4)
        x_4 = MaxPooling2D(pool_size=(max_length - 3, 1), name='pool_4')(x_4)

        
        x_5 = Conv2D(300, (5, 300), activation='relu', name='conv_5', padding='valid', kernel_regularizer='l2')(inputs)
#         x_5 = GlobalMaxPooling2D(name='pool_5')(x_5) 
        x_5 = MaxPooling2D(pool_size=(max_length - 4, 1), name='pool_5')(x_5)
    
            
        x_6 = Conv2D(300, (6, 300), activation='relu', name='conv_6', padding='valid', kernel_regularizer='l2')(inputs)
#         x_5 = GlobalMaxPooling2D(name='pool_5')(x_6) 
        x_6 = MaxPooling2D(pool_size=(max_length - 5, 1), name='pool_6')(x_6)

        x = Concatenate(name='concat')([x_3, x_4, x_5, x_6, lstm])
        x = Flatten(name='flatten')(x)
        x = Dropout(rate=.4, name='dropout')(x)
        x = Dense(500, activation='relu', name='dense')(x)
#         x = Dropout(rate=.2, name='dropout')(x)
        out = Dense(star_units, activation='softmax', name='stars')(x)
        return tf.keras.models.Model(inputs={'review':inputs}, outputs={'stars': out}, name='Blueberry')
        
    @tf.function
    def loss_fn(truth, logits):
        loss = tf.keras.losses.sparse_categorical_crossentropy(truth, logits)
        return loss

In [19]:
batch_size = 64
dataset, num_batches = make_dataset(batch_size=batch_size, embedded=True, to_ix=True, channelize=True)

TypeError: make_dataset() got an unexpected keyword argument 'batch_size'

In [20]:
def rename(newname):
    def decorator(f):
        f.__name__ = newname
        return f
    return decorator

In [21]:
def one_class_accuracy(class_ix):
    @rename('{}_acc'.format(class_ix+1))
    def single_class_accuracy(y_true, y_pred):
        class_id_true = K.cast(y_true, 'int64')
        class_id_preds = K.argmax(y_pred, axis=-1)
        # Replace class_id_preds with class_id_true for recall here
        accuracy_mask = K.cast(K.equal(class_id_preds, class_ix), 'int32')
        class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
        class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
        return K.cast(K.ones(batch_size, dtype='float64') * (class_acc / batch_size), dtype='float32')
    return single_class_accuracy

In [22]:
baby = BabyBlueberry.build(input_shape=(300,300,1))

In [23]:
adam = tf.keras.optimizers.Adam()

In [24]:
star_metrics = [one_class_accuracy(i) for i in range(5)]
star_metrics.append(tf.keras.metrics.sparse_categorical_accuracy)

In [25]:
# plot_model(baby, show_shapes=False, show_layer_names=True, to_file='conv-model.png',rankdir='LR' )

In [26]:
baby.compile(optimizer=adam, metrics={'stars': star_metrics} ,loss={'stars': BabyBlueberry.loss_fn})
log_dir = os.path.join(SRC_DIR, 'logs', 'CONV-{}'.format(datetime.fromtimestamp(time.time()).strftime('%H-%M-%S_%m-%d')))
tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=False, update_freq='batch')

In [27]:
for x, y in dataset['train']:break
truth = y['stars']
pred = baby(x['review'])['stars']

In [33]:
predictions, truth = [], []
for x, y in dataset['val'].take(10):
    pred = baby.predict_on_batch(x)
    truth.append(y['stars'].numpy())
    predictions.append(pred['stars'].numpy())

In [28]:
baby.fit(dataset['train'], epochs=10, validation_data= dataset['val'].take(100), callbacks=[tb], class_weight=class_weights)

Epoch 1/10
  49291/Unknown - 7687s 156ms/step - loss: 1.0619 - 1_acc: 0.1504 - 2_acc: 0.0721 - 3_acc: 0.1093 - 4_acc: 0.2192 - 5_acc: 0.4379 - sparse_categorical_accuracy: 0.6167

KeyboardInterrupt: 